In [11]:
import numpy as np
import pandas as pd
import gensim
import os
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
# GRU and LSTM are RNN it is sequence learning
from keras.layers import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model



In [12]:
data = pd.read_json("intent.json")
data

,ourIntents
0,"{'tag': 'predict', 'patterns': ['How can I kno..."
1,"{'tag': 'recommend', 'patterns': ['Recommend c..."
2,"{'tag': 'goodbye', 'patterns': ['bye', 'later'..."
3,"{'tag': 'name', 'patterns': ['what's your name..."
4,"{'tag': 'greeting', 'patterns': ['Hi', 'Hello'..."
5,"{'tag': 'weather', 'patterns': ['What's the we..."
6,"{'tag': 'stores', 'patterns': ['Where are the ..."
7,"{'tag': 'crops', 'patterns': ['Where are the f..."
8,"{'tag': 'friends', 'patterns': ['I want to mak..."
9,"{'tag': 'delete', 'patterns': ['I want to dele..."


In [13]:
dict = {}

In [14]:
cnt = 0
for i in range(0,len(data)):
    dict[data['ourIntents'][i]['tag']] = cnt;
    cnt = cnt+1

In [15]:
dict

{'predict': 0,
 'recommend': 1,
 'goodbye': 2,
 'name': 3,
 'greeting': 4,
 'weather': 5,
 'stores': 6,
 'crops': 7,
 'friends': 8,
 'delete': 9,
 'notfound': 10,
 'communication': 11}

In [16]:
text = []
y = []
for i in range(0,len(data)):
    sz = len(data['ourIntents'][i]['patterns'])
    for j in range(0,sz):
        text.append(data['ourIntents'][i]['patterns'][j])
        y.append(data['ourIntents'][i]['tag'])

In [17]:
text

['How can I know my yield?',
 'Can you predict my yield',
 'I want to know my production of crops',
 'What is the amount of crops will I get?',
 'I want to know production',
 'What is production going to be?',
 'Recommend crops for me',
 'Which crop should I grow?',
 'Any idea on which crop is best',
 'bye',
 'later',
 'see you',
 "what's your name?",
 'who are you?',
 'Hi',
 'Hello',
 'Hey',
 "What's the weather like?",
 'Is it rainy?',
 'How should I know my weather?',
 'Where are the nearby stores?',
 'Stores near me',
 'Where should I sell my crops',
 'Where are the farmers near me growing?',
 'What are the crops grown near me?',
 'Crops near me!',
 'I want to make new friends',
 'I want to talk to my friends',
 'Where are my friends?',
 'I want to delete my account',
 'I dislike you',
 'I fail to find crops near me when accessing the major crops',
 'I fail to find users near me at major crops feature',
 "I can't see users in the major crops",
 'No user is visible in the map',
 'I 

In [18]:
y = [dict[text] for text in y]

In [19]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def clean_text(text):
    all_reviews = []
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    for i in range(0,len(text)):
        now_text = text[i]
        try:
            now_text = now_text.lower()
            now_text = pattern.sub('', now_text)
            now_text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", now_text)
            now_text = re.sub(r"i'm", "i am", now_text)
            now_text = re.sub(r"he's", "he is", now_text)
            now_text = re.sub(r"she's", "she is", now_text)
            now_text = re.sub(r"that's", "that is", now_text)        
            now_text = re.sub(r"what's", "what is", now_text)
            now_text = re.sub(r"where's", "where is", now_text) 
            now_text = re.sub(r"\'ll", " will", now_text)  
            now_text = re.sub(r"\'ve", " have", now_text)  
            now_text = re.sub(r"\'re", " are", now_text)
            now_text = re.sub(r"\'d", " would", now_text)
            now_text = re.sub(r"\'ve", " have", now_text)
            now_text = re.sub(r"won't", "will not", now_text)
            now_text = re.sub(r"don't", "do not", now_text)
            now_text = re.sub(r"did't", "did not", now_text)
            now_text = re.sub(r"can't", "can not", now_text)
            now_text = re.sub(r"it's", "it is", now_text)
            now_text = re.sub(r"couldn't", "could not", now_text)
            now_text = re.sub(r"have't", "have not", now_text)
            tokens = word_tokenize(now_text)
            table = str.maketrans('', '', string.punctuation)
            stripped = [w.translate(table) for w in tokens]
            words = [word for word in stripped if word.isalpha()]
            
            PS = PorterStemmer()
            words = [PS.stem(w) for w in words ]
            words = ' '.join(words)
            all_reviews.append(words)
            print(words)
            cnt = cnt+1
        except:
            
            continue
    return all_reviews

all_reviews = clean_text(text)



how can i know my yield
can you predict my yield
i want to know my product of crop
what is the amount of crop will i get
i want to know product
what is product go to be
recommend crop for me
which crop should i grow
ani idea on which crop is best
bye
later
see you
what is your name
who are you
hi
hello
hey
what is the weather like
is it raini
how should i know my weather
where are the nearbi store
store near me
where should i sell my crop
where are the farmer near me grow
what are the crop grown near me
crop near me
i want to make new friend
i want to talk to my friend
where are my friend
i want to delet my account
i dislik you
i fail to find crop near me when access the major crop
i fail to find user near me at major crop featur
i can not see user in the major crop
no user is visibl in the map
i want to commun with a farmer
i want to find fertil


In [20]:
all_reviews

['how can i know my yield',
 'can you predict my yield',
 'i want to know my product of crop',
 'what is the amount of crop will i get',
 'i want to know product',
 'what is product go to be',
 'recommend crop for me',
 'which crop should i grow',
 'ani idea on which crop is best',
 'bye',
 'later',
 'see you',
 'what is your name',
 'who are you',
 'hi',
 'hello',
 'hey',
 'what is the weather like',
 'is it raini',
 'how should i know my weather',
 'where are the nearbi store',
 'store near me',
 'where should i sell my crop',
 'where are the farmer near me grow',
 'what are the crop grown near me',
 'crop near me',
 'i want to make new friend',
 'i want to talk to my friend',
 'where are my friend',
 'i want to delet my account',
 'i dislik you',
 'i fail to find crop near me when access the major crop',
 'i fail to find user near me at major crop featur',
 'i can not see user in the major crop',
 'no user is visibl in the map',
 'i want to commun with a farmer',
 'i want to find fe

In [21]:
from sklearn.model_selection import train_test_split
validation_split = 1
max_length = 15

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(all_reviews)
sequences = tokenizer_obj.texts_to_sequences(all_reviews)

word_index = tokenizer_obj.word_index
print("unique tokens - "+str(len(word_index)))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab_size - '+str(vocab_size))
# This is for the values which are not present in the dataset

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')


X_train = lines_pad
y_train = y

unique tokens - 76
vocab_size - 77


In [22]:
from joblib import dump
dump(tokenizer_obj,"tokenizer.joblib")

['tokenizer.joblib']

In [23]:
X_train

array([[22, 15,  1, 12,  4, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [15, 13, 33,  4, 23,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  6,  3, 12,  4, 16, 24,  2,  0,  0,  0,  0,  0,  0,  0],
       [10,  7,  5, 34, 24,  2, 35,  1, 36,  0,  0,  0,  0,  0,  0],
       [ 1,  6,  3, 12, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10,  7, 16, 37,  3, 38,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [39,  2, 40,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [25,  2, 17,  1, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [41, 42, 43, 25,  2,  7, 44,  0,  0,  0,  0,  0,  0,  0,  0],
       [45,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [46,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [27, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [10,  7, 47, 48,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [49, 11, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [50,  0,  0,  0,  0,  0,  0

In [24]:
X_train.shape

(37, 15)

In [25]:
y_train = np.array(y_train)
y_train.shape

(37,)

In [26]:
embedding_index = {}
embedding_dim = 300
glove_dir = "D:\GloveModel"
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'), encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coeff
f.close()


In [27]:
# Mapping the word  from vector
embedding_dim = 300
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [29]:


embedding_layer = Embedding(len(word_index) + 1,
                                embedding_dim,
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=False)

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
from keras.utils import to_categorical


y_train = to_categorical(y_train)

from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.layers import Flatten
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU


model_glove = Sequential()
model_glove.add(embedding_layer)
model_glove.add(Bidirectional(LSTM(units=50)))
model_glove.add(Dense(12, activation='softmax'))

model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_glove.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 300)           23100     
                                                                 
 bidirectional (Bidirectiona  (None, 100)              140400    
 l)                                                              
                                                                 
 dense (Dense)               (None, 12)                1212      
                                                                 
Total params: 164,712
Trainable params: 141,612
Non-trainable params: 23,100
_________________________________________________________________
None


In [ ]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

model_glove.fit(X_train, y_train, epochs=100,verbose=1,callbacks = [callback])


Epoch 1/100
2/2 [==============================] - 5s 38ms/step - loss: 2.4659 - accuracy: 0.0541
Epoch 2/100
2/2 [==============================] - 0s 13ms/step - loss: 2.3660 - accuracy: 0.2973
Epoch 3/100
2/2 [==============================] - 0s 12ms/step - loss: 2.2938 - accuracy: 0.3243
Epoch 4/100
2/2 [==============================] - 0s 15ms/step - loss: 2.2228 - accuracy: 0.3784
Epoch 5/100
2/2 [==============================] - 0s 12ms/step - loss: 2.1564 - accuracy: 0.3784
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 2.0927 - accuracy: 0.4595
Epoch 7/100
2/2 [==============================] - 0s 9ms/step - loss: 2.0307 - accuracy: 0.4324
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 1.9792 - accuracy: 0.4054
Epoch 9/100
2/2 [==============================] - 0s 13ms/step - loss: 1.9194 - accuracy: 0.4054
Epoch 10/100
2/2 [==============================] - 0s 12ms/step - loss: 1.8593 - accuracy: 0.4324
Epoch 11/100
2/2 [====

In [ ]:
def predict_sarcasm(text):
    # Send an array of 1 value
    test_lines = clean_text(text)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    print(test_review_pad)
    pred = model_glove.predict(test_review_pad)
    pred*=100
    print(pred)
    return pred

In [ ]:
dictionary2 = {}
for i in range(0,len(dict)):
    tag = data["ourIntents"][i]['tag']
    sz = len(data["ourIntents"][i]['responses'])
    res = []
    for j in range(0,sz):
        res.append(data['ourIntents'][i]['responses'][j])
    dictionary2[tag] = res

dictionary2

{'predict': ['You can go to our homepage and click on get started 🤝'],
 'recommend': ['Congrats! You accessed one of to-be-developed features. I am sincerely sorry to tell that the feature is not yet available',
  'Features is in development stage. Inconvience regretted'],
 'goodbye': ['Bye', 'take care'],
 'name': ['My name is farm sensei. Developers named me that',
  'I am farm sensei. Ready to be at your service'],
 'greeting': ['Hi there. I am Farm Sensei!. Ready to help', 'Hello', 'Hi :)'],
 'weather': ['Go to homepage and access weather forecast for our amazing features.'],
 'stores': ['Farm sensei to the rescue! You can access stores from the homepage!',
  'To know the stores near you can use our stores feature in the homepage'],
 'crops': ['Farm sensei to the rescue! You can access crops from the homepage!',
  'To know the crops grown near you can use our major crops feature in the homepage 🚀'],
 'friends': ['Farm sensei to the rescue! Click on your profile on the toppage! Ther

In [1]:
import random
from textblob import TextBlob
text ="what are the crops grown near me"
text = str(TextBlob(text).correct())
pred  =predict_sarcasm([text])
pred[0] = np.array(pred[0])
i = np.argmax(pred[0])
inverse_dict = {value: key for key, value in dict.items()}

print(inverse_dict[i])
ourResult = random.choice(dictionary2[inverse_dict[i]])
print(ourResult)

NameError: name 'predict_sarcasm' is not defined

In [13]:
# # model_glove.save("chatbotmodel.h5")
# from tensorflow.keras.models import load_model
# # model_glove = load("chatbotmodel.joblib")
# model_glove = load_model('chatbotmodel.h5')

KeyError: "Registering two gradient with name 'ReduceDataset'! (Previous registration was in register C:\\Users\\hp\\AppData\\Roaming\\Python\\Python311\\site-packages\\tensorflow\\python\\framework\\registry.py:65)"

KeyError: "Registering two gradient with name 'ReduceDataset'! (Previous registration was in register C:\\Users\\hp\\AppData\\Roaming\\Python\\Python311\\site-packages\\tensorflow\\python\\framework\\registry.py:65)"